# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Importing Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# checking the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. ETL Pipeline for Processing files into Apache Cassandra database

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [6]:
# Creating a connection to a Cassandra cluster
from cassandra.cluster import Cluster
cluster = Cluster()
# Session to establish connection and begin executing queries
session = cluster.connect()

#### Create Keyspace

In [7]:
# Creating a Keyspace 
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra we model the database tables on the queries we want to run.

## We will create queries for the following three user stories:

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


#### Query 1:

##### The music_app_session table is partitioned by session_id with item_in_session as clustering column to make the primary key unique.

In [9]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS music_app_session "
query = query + "(session_id INT, item_in_session INT, artist TEXT, song TEXT, length FLOAT, PRIMARY KEY(session_id, item_in_session))"
session.execute(query)
                    

In [10]:
# file for denormalized dataset
file = 'event_datafile_new.csv'
# open file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
## INSERT data into tables
        query = "INSERT INTO music_app_session (session_id, item_in_session, artist, song, length)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [11]:
## SELECT statement for Query 1
query = "SELECT session_id, item_in_session, artist, song, length FROM music_app_session WHERE session_id = 338 AND item_in_session = 4"
rows = session.execute(query)
    
for row in rows:
    print (row.session_id, row.item_in_session, row.artist, row.song, row.length)

338 4 Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Query 2:

##### The music_app_user table has a partition key of user_id with clustering columns session_id and item_in_session. This allows result to be sorted by itemInSession.

In [12]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS music_app_user "
query = query + "(user_id INT, session_id INT, item_in_session INT, artist TEXT, song TEXT, first_name TEXT, last_name TEXT, PRIMARY KEY(user_id, session_id, item_in_session))"
session.execute(query)

# open file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT data into tables
        query = "INSERT INTO music_app_user (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + " VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        
## SELECT statement for Query 2
query = "SELECT user_id, session_id, item_in_session, artist, song, first_name, last_name FROM music_app_user WHERE user_id = 10 AND session_id = 182"
rows = session.execute(query)
    
for row in rows:
    print (row.user_id, row.session_id, row.item_in_session, row.artist, row.song, row.first_name, row.last_name)

10 182 0 Down To The Bone Keep On Keepin' On Sylvie Cruz
10 182 1 Three Drives Greece 2000 Sylvie Cruz
10 182 2 Sebastien Tellier Kilometer Sylvie Cruz
10 182 3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Query 3:

##### The music_app_song table is partitioned by song with user_id as clustering column to make the primary key unique.

In [13]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS music_app_song "
query = query + "(song TEXT, user_id INT, user TEXT, PRIMARY KEY(song, user_id))"
session.execute(query)

# open file
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERT data into tables
        query = "INSERT INTO music_app_song (song, user_id, user)"
        query = query + " VALUES(%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1] + " " + line[4]))
        
## SELECT statement for Query 3
query = "SELECT user, song FROM music_app_song WHERE song='All Hands Against His Own'"
rows = session.execute(query)
    
for row in rows:
    print (row.user, row.song)

Jacqueline Lynch All Hands Against His Own
Tegan Levine All Hands Against His Own
Sara Johnson All Hands Against His Own


### Drop the tables before closing out the sessions

In [14]:
# Drop tables
session.execute("DROP TABLE IF EXISTS music_app_session")
session.execute("DROP TABLE IF EXISTS music_app_user")
session.execute("DROP TABLE IF EXISTS music_app_song")

### Close the session and cluster connection¶

In [15]:
# close session and cluster connection
session.shutdown()
cluster.shutdown()